# Phase 3: Complex Architects: RNNs,CNNs,Transformer,...

# Transformer part

Tên và MSSV của từng thành viên:
- Đinh Viết Lợi - 22120188.
- Nguyễn Trần Lợi - 22120190.
- Nguyễn Nhật Long - 22120194.


## Nắm yêu cầu của Phase 3:
Đây là bài toán phân loại nhận biết đoạn âm thanh có chứa tiếng gà tây hay không bằng cách sử dụng  và tuân theo quy trình: Phân tích Dữ liệu Khám phá (Exploratory Data Analysis), Phát triển Mô hình (Model Development) và Đánh giá Mô hình (Model Evaluation).

## Import các thư viện cần thiết

In [1]:
import time
import tracemalloc
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, Layer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras_nlp
import warnings
warnings.filterwarnings('ignore')
import tqdm as notebook_tqdm

c:\Users\pc\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Đọc dữ liệu

In [2]:
with open('Datasets/train.json', 'r') as f:
    train_data = json.load(f)
with open('Datasets/test.json', 'r') as f:
    test_data = json.load(f)
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

In [3]:
train_data.head()

,audio_embedding,is_turkey,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",0,kDCk3hLIVXo,70,60
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",1,DPcGzqHoo7Y,40,30
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",1,7yM63MTHh5k,240,230
3,"[[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...",1,luG3RmUAxxM,520,510
4,"[[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...",0,PIm3cjxTpOk,10,0


In [4]:
test_data.head()

,audio_embedding,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...",pyKh38FXD3E,10,0
1,"[[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...",THhP1idrWXA,40,30
2,"[[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...",jsw3T6GY2Nw,40,30
3,"[[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...",nFkXTMHcjMU,24,14
4,"[[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...",Au8g9kAlrLQ,40,30


## Khám phá dữ liệu

### Các cột có trong dữ liệu

In [5]:
train_data.columns

Index(['audio_embedding', 'is_turkey', 'vid_id',
       'end_time_seconds_youtube_clip', 'start_time_seconds_youtube_clip'],
      dtype='object')

- Dữ liệu trong `train_df` chứa các cột trên. Trong đó, các cột cần thiết để thực hiện huấn luyện là `audio_embedding`, `is_turkey`.

In [6]:
test_data.columns

Index(['audio_embedding', 'vid_id', 'end_time_seconds_youtube_clip',
       'start_time_seconds_youtube_clip'],
      dtype='object')

- `test_df` có các cột trên. Dựa vào yêu cầu, các cột sử dụng là `vid_id`, `audio_embedding`.

## Phát triển mô hình

### Xử lý dữ liệu trước khi huấn luyện

#### Chuyển list thành mảng numpy 

In [7]:
train_embeddings = train_data['audio_embedding'].apply(np.array)

#### Tìm chiều dài tối đa và số chiều embedding

In [8]:
max_len = train_embeddings.apply(len).max()
embedding_dim = len(train_embeddings.iloc[0][0])

#### Đảm bảo các vector có cùng độ dài bằng padding 

In [9]:
train_X = pad_sequences(train_embeddings, maxlen=max_len, dtype='float32', padding='post', truncating='post')
train_Y = train_data['is_turkey'].values

#### Loại bỏ các list rỗng

In [10]:
valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
test_embeddings = test_data.loc[valid_idx, 'audio_embedding'].apply(np.array)

#### Chuẩn hóa embedding của tập Test 

In [11]:
test_X = pad_sequences(test_embeddings, maxlen=max_len, dtype='float32', padding='post', truncating='post')

### Lấy kích thước của dữ liệu đầu vào

#### B: số mẫu, T: độ dài chuỗi, D: số chiều embedding

In [12]:
B, T, D = train_X.shape 

### Chuẩn hóa dữ liệu bằng Standard Scaler

In [13]:
scaler = StandardScaler() 
train_X = scaler.fit_transform(train_X.reshape(-1, D)).reshape(B, T, D)
test_X = scaler.transform(test_X.reshape(-1, D)).reshape(test_X.shape[0], T, D)

### Tạo lớp Positional Encoding 

In [14]:
class PositionalEncoding(Layer):
    def __init__(self):
        super(PositionalEncoding, self).__init__()

    def call(self, x):
        seq_len = tf.shape(x)[1]
        d_model = tf.shape(x)[2]

        pos = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]
        i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]

        angle_rates = 1 / tf.pow(10000., (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        angle_rads = pos * angle_rates

        sines = tf.sin(angle_rads[:, 0::2])
        cosines = tf.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]

        return x + pos_encoding

### Xây dựng mô hình Transformer: 

In [15]:
def build_transformer(seq_len, d_model):
    inputs = Input(shape=(seq_len, d_model))
    x = PositionalEncoding()(inputs)

    for _ in range(2):
        x = keras_nlp.layers.TransformerEncoder(
            intermediate_dim=256,
            num_heads = 8 ,
            dropout=0.2,
        )(x)

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["AUC"])
    return model

### Khởi tạo danh sách các mô hình tối ưu nhất

In [16]:
qualified_models = []

### Huấn luyện, model thỏa tiêu chí được lưu lại, in ra thời gian, bộ nhớ và các chỉ số. 

##### Vì kết quả mỗi lần huấn luyện model là khác nhau, ta huấn luyện model 200 lần để xem model nào cho thông số cao nhất thì lưu lại

##### **Nhận thấy:** Khi **Precision** và **Accuracy** từ 0.94 trở lên thì **Score** thường đạt được khá cao. Đặc biệt là **Precision** càng cao thì **Score** thường càng cao. Do đó ta chỉ lưu lại những model thỏa điều này.

In [17]:
for i in range(200):
    start_time = time.time()
    tracemalloc.start()
    print(f"\n--- Training iteration {i} ---")
    train_X_, val_X, train_Y_, val_Y = train_test_split(train_X, train_Y, test_size=0.3, random_state=45)

    model = build_transformer(seq_len=max_len, d_model=embedding_dim)
    early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=0)

    model.fit(train_X_, train_Y_,
              validation_data=(val_X, val_Y),
              epochs=80,
              batch_size=32,
              callbacks=[early_stop],
              verbose=0)

    y_pred_prob = model.predict(val_X, verbose=0).ravel()
    y_pred = (y_pred_prob >= 0.5).astype(int)

    precision = precision_score(val_Y, y_pred)
    recall = recall_score(val_Y, y_pred)
    auc = roc_auc_score(val_Y, y_pred_prob)
    acc = accuracy_score(val_Y, y_pred)
    f1 = f1_score(val_Y, y_pred)

    # Ghi nhận thời gian và bộ nhớ sau huấn luyện
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    elapsed_time = end_time - start_time
    peak_memory_MB = peak / (1024 * 1024)

    if precision >= 0.94 and recall >= 0.94 and auc >= 0.99 and f1 >= 0.94 and acc >= 0.94:
        qualified_models.append({
            'model': model,
            'auc': auc,
            'accuracy': acc,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'time': elapsed_time,
            'memory_MB': peak_memory_MB
        })


--- Training iteration 0 ---


--- Training iteration 1 ---

--- Training iteration 2 ---

--- Training iteration 3 ---

--- Training iteration 4 ---

--- Training iteration 5 ---

--- Training iteration 6 ---

--- Training iteration 7 ---

--- Training iteration 8 ---

--- Training iteration 9 ---

--- Training iteration 10 ---

--- Training iteration 11 ---

--- Training iteration 12 ---

--- Training iteration 13 ---

--- Training iteration 14 ---

--- Training iteration 15 ---

--- Training iteration 16 ---

--- Training iteration 17 ---

--- Training iteration 18 ---

--- Training iteration 19 ---

--- Training iteration 20 ---

--- Training iteration 21 ---

--- Training iteration 22 ---

--- Training iteration 23 ---

--- Training iteration 24 ---

--- Training iteration 25 ---

--- Training iteration 26 ---

--- Training iteration 27 ---

--- Training iteration 28 ---

--- Training iteration 29 ---

--- Training iteration 30 ---

--- Training iteration 31 ---

--- Training ite

### In ra các mô hình thỏa tiêu chí

In [18]:
# Sắp xếp mô hình
qualified_models = sorted(qualified_models, key=lambda x: -x['precision'])

# In kết quả
if qualified_models:
    print("\n=== Top models with precision >= 0.94 and recall >= 0.94 ===")
    for i, m in enumerate(qualified_models):
        print(f"[Model {i}]")
        print(f"  AUC       : {m['auc']:.4f}")
        print(f"  Accuracy  : {m['accuracy']:.4f}")
        print(f"  Precision : {m['precision']:.4f}")
        print(f"  Recall    : {m['recall']:.4f}")
        print(f"  F1 Score  : {m['f1']:.4f}")
        print(f"  ⏱ Time       : {m['time']:.2f} sec")
        print(f"  📦 Memory     : {m['memory_MB']:.2f} MB")
else:
    print("Không có mô hình nào đạt các tiêu chí >= 0.94.")


=== Top models with precision >= 0.94 and recall >= 0.94 ===
[Model 0]
  AUC       : 0.9951
  Accuracy  : 0.9721
  Precision : 0.9784
  Recall    : 0.9510
  F1 Score  : 0.9645
  ⏱ Time       : 18.34 sec
  📦 Memory     : 27.56 MB
[Model 1]
  AUC       : 0.9921
  Accuracy  : 0.9666
  Precision : 0.9712
  Recall    : 0.9441
  F1 Score  : 0.9574
  ⏱ Time       : 19.24 sec
  📦 Memory     : 27.53 MB
[Model 2]
  AUC       : 0.9948
  Accuracy  : 0.9666
  Precision : 0.9712
  Recall    : 0.9441
  F1 Score  : 0.9574
  ⏱ Time       : 42.43 sec
  📦 Memory     : 27.17 MB
[Model 3]
  AUC       : 0.9935
  Accuracy  : 0.9721
  Precision : 0.9650
  Recall    : 0.9650
  F1 Score  : 0.9650
  ⏱ Time       : 20.97 sec
  📦 Memory     : 27.21 MB
[Model 4]
  AUC       : 0.9938
  Accuracy  : 0.9666
  Precision : 0.9645
  Recall    : 0.9510
  F1 Score  : 0.9577
  ⏱ Time       : 19.13 sec
  📦 Memory     : 27.55 MB
[Model 5]
  AUC       : 0.9924
  Accuracy  : 0.9666
  Precision : 0.9645
  Recall    : 0.9510
  F1

### Lựa chọn mô hình để dự đoán cho tập test

#### Mặc định là dùng model có precision cao nhất 

In [19]:
if qualified_models:
    best_model = qualified_models[0]['model']
    test_pred_prob = best_model.predict(test_X, verbose=0).ravel()
    test_data['is_turkey'] = -1.0
    test_data.loc[valid_idx, 'is_turkey'] = test_pred_prob
    test_data.loc[valid_idx, ['vid_id', 'is_turkey']].to_csv('result.csv', index=False)
    print("\n✅ Đã lưu kết quả dự đoán từ mô hình tốt nhất vào 'result.csv'")


✅ Đã lưu kết quả dự đoán từ mô hình tốt nhất vào 'result.csv'
